In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from ipywidgets import interact, FloatSlider

## Raw SVI parametrization
$$ k = \log(\frac{K}{S_0e^{rT}}) $$
$$ w(k) = a + b(\rho(k - m) + \sqrt{(k - m)^2 + \sigma^2})$$
$$ w(k) = \sigma_{BS}^2T

In [2]:
@dataclass
class raw_SVI_params:
    a: float
    b: float
    m: float
    sigma: float
    rho: float


In [5]:
def plot_BS_implied_volatility(S0, T, r, a, b, m, sigma, rho, K_min = 30 , K_max = 180):
    p = raw_SVI_params(a = a, b = b, m = m, sigma = sigma, rho = rho)
    # strikes 
    K = np.linspace(K_min, K_max, 200)
    k = np.log(K/(S0 * np.exp(r*T)))
    w_k = p.a + p.b * (p.rho * (k - p.m) + np.sqrt((k - p.m) ** 2 + p.sigma ** 2))
    implied_BS_vols = np.sqrt(w_k / T) * 100
    fig, ax = plt.subplots()
    ax.scatter(k, implied_BS_vols, color = 'black')
    ax.set_xlabel("Log-moneyness")
    ax.set_ylabel("BS implied volatility (%)")
    ax.set_title("BS implied volatility(%) for raw SVI model")
    ax.grid(True)
    plt.show()

interact(plot_BS_implied_volatility,
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=1.0, min=0.05, max=5.0, step=0.05, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         a=FloatSlider(value=1.5, min=0.01, max=10.0, step=0.05, description="a"),
         b=FloatSlider(value=0.12, min=0.001, max=0.5, step=0.005, description="b"),
         m=FloatSlider(value=0.01, min=0.01, max=2.0, step=0.01, description="m"),
         sigma=FloatSlider(value=0.1, min=0, max=0.3, step=0.01, description="sigma"),
         rho=FloatSlider(value=0.1, min=-1.0, max=1.0, step=0.01, description="rho"),

        )

interactive(children=(FloatSlider(value=100.0, description='S0', max=300.0, min=20.0, step=1.0), FloatSlider(v…

<function __main__.plot_BS_implied_volatility(S0, T, r, a, b, m, sigma, rho, K_min=30, K_max=180)>

In [ ]:
def plot_risk_neutral_density_SVI(S0, T, r, a, b, m, sigma, rho,
                                  K_min=30, K_max=180):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    p = raw_SVI_params(a=a, b=b, m=m, sigma=sigma, rho=rho)

    K = np.linspace(K_min, K_max, 400)
    F = S0 * np.exp(r * T)
    k = np.log(K / F)

    # Total variance w(k)
    x = k - p.m
    w = p.a + p.b * (p.rho * x + np.sqrt(x**2 + p.sigma**2))

    # d1, d2 in terms of total variance w and log-moneyness k
    sqrt_w = np.sqrt(w)
    d1 = -k / sqrt_w + 0.5 * sqrt_w
    d2 = -k / sqrt_w - 0.5 * sqrt_w

    # First and second derivatives of w with respect to k
    w_k  = p.b * (p.rho + x / np.sqrt(x**2 + p.sigma**2))
    w_kk = p.b * (p.sigma**2) / (x**2 + p.sigma**2)**1.5

    # g(k): butterfly condition factor
    g_k = (1.0 - k * w_k / (2.0 * w))**2 \
          - 0.25 * w_k**2 * (1.0 / w + 0.25) \
          + 0.5 * w_kk

    # Risk-neutral density of S_T at K
    f_x = g_k / (K * np.sqrt(2 * np.pi * w)) * np.exp(-0.5 * d2**2)

    # Plot g(k)
    ax[0].plot(K, g_k, color='black')
    ax[0].set_xlabel("Strike")
    ax[0].set_ylabel("g(k)")
    ax[0].set_title("g(k) against Strike")

    # Plot density f_{S_T}(K)
    ax[1].plot(K, f_x, color='black')
    ax[1].set_xlabel("Stock price")
    ax[1].set_ylabel(r"$f_{S_{T}}(K)$")
    ax[1].axvline(x=S0, color='red', linestyle='--', linewidth=2)
    ax[1].set_title("Risk-neutral density for raw SVI model")

    plt.tight_layout()
    plt.show()

interact(plot_risk_neutral_density_SVI,
         S0=FloatSlider(value=100, min=20,  max=300, step=1,   description="S0"),
         T=FloatSlider(value=0.55, min=0.05, max=5.0, step=0.05, description="T"),
         r=FloatSlider(value=0.02, min=0.0, max=0.1, step=0.002, description="r"),
         a=FloatSlider(value=0.01, min=-0.2, max=10.0, step=0.01, description="a"),
         b=FloatSlider(value=0.03, min=0.001, max=0.5, step=0.005, description="b"),
         m=FloatSlider(value=0.50, min=0.01, max=2.0, step=0.01, description="m"),
         sigma=FloatSlider(value=0.06, min=0, max=0.8, step=0.01, description="sigma"),
         rho=FloatSlider(value=0.28, min=-1.0, max=1.0, step=0.01, description="rho"))


interactive(children=(FloatSlider(value=100.0, description='S0', max=300.0, min=20.0, step=1.0), FloatSlider(v…

<function __main__.plot_risk_neutral_density_SVI(S0, T, r, a, b, m, sigma, rho, K_min=30, K_max=180)>